#We will fine-tune a babbage-002 classifier (replacement for the ada models) to distinguish between the two sports: Baseball and Hockey.



In [1]:
!pip install --upgrade openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.8 MB/s eta 0:00:00


In [4]:
from google.colab import userdata
OPEN_AI_KEY=userdata.get('opeaikey4o')

In [5]:
from openai import OpenAI
import matplotlib.pyplot as plt
import time
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import openai
import os

## Set the API key and model name
client = OpenAI(api_key=OPEN_AI_KEY)

In [ ]:
# categories = ['rec.sport.baseball', 'rec.sport.hockey']
# sports_dataset = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories)
#colab might not work

In [6]:
sports_dataset=pd.read_csv('20_newsgroup_sports.csv')

In [7]:
sports_dataset.head()

,Unnamed: 0,text,target,title,date
0,0,From: dougb@comm.mot.com (Doug Bank)\nSubject:...,0,rec.sport.baseball,2024-06-22 10:24:26.596317
1,1,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...,1,rec.sport.hockey,2024-06-22 10:24:26.596317
2,2,From: rudy@netcom.com (Rudy Wade)\nSubject: Re...,0,rec.sport.baseball,2024-06-22 10:24:26.596317
3,3,From: monack@helium.gas.uug.arizona.edu (david...,1,rec.sport.hockey,2024-06-22 10:24:26.596317
4,4,Subject: Let it be Known\nFrom: <ISSBTL@BYUVM....,0,rec.sport.baseball,2024-06-22 10:24:26.596317


# Data exploration

In [8]:
print(sports_dataset['text'][0])

From: dougb@comm.mot.com (Doug Bank)
Subject: Re: Info needed for Cleveland tickets
Reply-To: dougb@ecs.comm.mot.com
Organization: Motorola Land Mobile Products Sector
Distribution: usa
Nntp-Posting-Host: 145.1.146.35
Lines: 17

In article <1993Apr1.234031.4950@leland.Stanford.EDU>, bohnert@leland.Stanford.EDU (matthew bohnert) writes:

|> I'm going to be in Cleveland Thursday, April 15 to Sunday, April 18.
|> Does anybody know if the Tribe will be in town on those dates, and
|> if so, who're they playing and if tickets are available?

The tribe will be in town from April 16 to the 19th.
There are ALWAYS tickets available! (Though they are playing Toronto,
and many Toronto fans make the trip to Cleveland as it is easier to
get tickets in Cleveland than in Toronto.  Either way, I seriously
doubt they will sell out until the end of the season.)

-- 
Doug Bank                       Private Systems Division
dougb@ecs.comm.mot.com          Motorola Communications Sector
dougb@nwu.edu       

In [9]:
print(sports_dataset['text'][1])

From: gld@cunixb.cc.columbia.edu (Gary L Dare)
Subject: Re: Flames Truly Brutal in Loss
Nntp-Posting-Host: cunixb.cc.columbia.edu
Reply-To: gld@cunixb.cc.columbia.edu (Gary L Dare)
Organization: PhDs In The Hall
Distribution: na
Lines: 13


This game would have been great as part of a double-header on ABC or
ESPN; the league would have been able to push back-to-back wins by
Le Magnifique and The Great One.  Unfortunately, the only network
that would have done that was SCA, seen in few areas and hard to
justify as a pay channel. )-;

gld
--
~~~~~~~~~~~~~~~~~~~~~~~~ Je me souviens ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Gary L. Dare
> gld@columbia.EDU 			GO  Winnipeg Jets  GO!!!
> gld@cunixc.BITNET			Selanne + Domi ==> Stanley



trim the text colum

In [10]:
sports_dataset['text']=sports_dataset['text'].str.slice(0,800)

In [11]:
print(sports_dataset['text'][0])

From: dougb@comm.mot.com (Doug Bank)
Subject: Re: Info needed for Cleveland tickets
Reply-To: dougb@ecs.comm.mot.com
Organization: Motorola Land Mobile Products Sector
Distribution: usa
Nntp-Posting-Host: 145.1.146.35
Lines: 17

In article <1993Apr1.234031.4950@leland.Stanford.EDU>, bohnert@leland.Stanford.EDU (matthew bohnert) writes:

|> I'm going to be in Cleveland Thursday, April 15 to Sunday, April 18.
|> Does anybody know if the Tribe will be in town on those dates, and
|> if so, who're they playing and if tickets are available?

The tribe will be in town from April 16 to the 19th.
There are ALWAYS tickets available! (Though they are playing Toronto,
and many Toronto fans make the trip to Cleveland as it is easier to
get tickets in Cleveland than in Toronto.  Either way, I seriously



In [12]:
sports_dataset['title'][0]

'rec.sport.baseball'

In [13]:
len_all,len_baseball,len_hocker=len(sports_dataset),len(sports_dataset[sports_dataset['target']==0]),len(sports_dataset[sports_dataset['target']==1])

print(f"Total examples:{len_all} ,Baseball example {len_baseball} ,Hockey example {len_hocker}")

Total examples:1197 ,Baseball example 597 ,Hockey example 600


#Data Preparation
We transform the dataset into a pandas dataframe, with a column for prompt and completion. The prompt contains the email from the mailing list, and the completion is a name of the sport, either hockey or baseball. For demonstration purposes only and speed of fine-tuning we take only 300 examples. In a real use case the more examples the better the performance

In [14]:
df=pd.DataFrame(columns=['text','title'],data=sports_dataset.sample(n=100,random_state=42))
df.head()


,text,title
921,From: dtate+@pitt.edu (David M. Tate)\nSubject...,rec.sport.baseball
321,From: golchowy@alchemy.chem.utoronto.ca (Geral...,rec.sport.hockey
101,From: kenney@tribe.b17d.ingr.COM (David Kenney...,rec.sport.baseball
920,From: cmk@athena.mit.edu (Charles M Kozierok)\...,rec.sport.baseball
58,From: filinuk@staff.dccs.upenn.edu (Geoff Fili...,rec.sport.hockey


In [17]:
len_all,len_baseball,len_hocker=len(df),len(df[df['title']=='rec.sport.baseball']),len(df[df['title']=='rec.sport.hockey'])

print(f"Total examples:{len_all} ,Baseball example {len_baseball} ,Hockey example {len_hocker}")

Total examples:100 ,Baseball example 50 ,Hockey example 50


In [15]:
len(df)

100

Change column name

In [18]:
df=df.rename(columns={'text':'prompt','title':'completion'})
df.head()

,prompt,completion
921,From: dtate+@pitt.edu (David M. Tate)\nSubject...,rec.sport.baseball
321,From: golchowy@alchemy.chem.utoronto.ca (Geral...,rec.sport.hockey
101,From: kenney@tribe.b17d.ingr.COM (David Kenney...,rec.sport.baseball
920,From: cmk@athena.mit.edu (Charles M Kozierok)\...,rec.sport.baseball
58,From: filinuk@staff.dccs.upenn.edu (Geoff Fili...,rec.sport.hockey


Both baseball and hockey are single tokens. We save the dataset as a jsonl file.

Note:
The value orient='records' is one of these options. When you use this orientation, the DataFrame is converted into a list of JSON objects, with each JSON object corresponding to a row in the DataFrame. This format is particularly useful for creating JSON outputs that are easy to read and work with, especially when interacting with APIs or exporting data for use in other systems.

In [19]:
df.to_json("sport2.jsonl",orient='records',lines=True)

## Data Preparation tool
We can now use a data preparation tool which will suggest a few improvements to our dataset before fine-tuning. Before launching the tool we update the openai library to ensure we're using the latest data preparation tool. We additionally specify -q which auto-accepts all suggestions.



In [20]:
!openai tools fine_tunes.prepare_data -f sport2.jsonl -q

Analyzing...

- Your file contains 100 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- All completions start with prefix `rec.sport.`. Most of the time you should only add the output data into the completion, without any prefix
- The completion should start with a whitespace character (` `). This tend

In [21]:
train_file=client.files.create(file=open("sport2_prepared_train.jsonl", "rb"),purpose='fine-tune')
valid_file=client.files.create(file=open("sport2_prepared_valid.jsonl", "rb"),purpose='fine-tune')


# Part 2
# Fine Tune Classfication data

In [22]:
fine_tuning_job=client.fine_tuning.jobs.create(training_file=train_file.id,validation_file=valid_file.id,model="babbage-002")
print(fine_tuning_job)

FineTuningJob(id='ftjob-WpqRhEvBZzSRSnbP33W2AkL9', created_at=1720256238, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='babbage-002', object='fine_tuning.job', organization_id='org-PRq3tYU2rFVBkygKcRdWRjwB', result_files=[], seed=117044035, status='validating_files', trained_tokens=None, training_file='file-j1vm7REmxADc3jXQeIowx5Pi', validation_file='file-mAxQaWBgzIxcAioB1jjHux8d', estimated_finish=None, integrations=[], user_provided_suffix=None)


The model is successfully trained in about ten minutes. You can watch the finetune happen on https://platform.openai.com/finetune/

You can also check on its status programatically:

In [23]:
retrieved_jobs=client.fine_tuning.jobs.retrieve(fine_tuning_job.id)
status=retrieved_jobs.status
print(status)

running


In [5]:
retrieved_jobs.error

In [24]:
while True:
  time.sleep(10)
  retrieved_jobs=client.fine_tuning.jobs.retrieve(fine_tuning_job.id)
  status=retrieved_jobs.status
  print(status)
  if(status=='succeeded'):
    break

succeeded


## Using the model

We can now call the model to get the predictions.




In [25]:
test = pd.read_json('sport2_prepared_valid.jsonl', lines=True)
test.head()

,prompt,completion
0,From: golchowy@alchemy.chem.utoronto.ca (Geral...,hockey
1,From: kenney@tribe.b17d.ingr.COM (David Kenney...,baseball
2,From: dmoney@magnus.acs.ohio-state.edu (Dean R...,hockey
3,From: awe@loch.mit.edu (Ari Epstein)\nSubject:...,baseball
4,From: binkley@let.rug.nl (P.A. Binkley)\nSubje...,baseball


In [26]:
test['prompt'][0]

"From: golchowy@alchemy.chem.utoronto.ca (Gerald Olchowy)\nSubject: Re: Panther's President\nOrganization: University of Toronto Chemistry Department\nLines: 20\n\nIn article <0P6a3B1w165w@cybernet.cse.fau.edu> jimg@cybernet.cse.fau.edu (Jim Gorycki) writes:\n>\n>A little Bio from _Sun-Sentinel_\n>Torrey, the architect of four consecutive Stanley Cup champions as \n>persident and general manager of the New York Islanders.\n>Throughout his 27 years in the NHL, Bill Torrey's bow ties have become\n>as much of a signature as Andre Agassi's hair.\n>\n>The Panthers will introduce a uniform, insignia, and ticket-price \n>information in early next month.  In the meantime, Huizenga leaves the\n>day-to-day operation in the hands of Torrey and Bob Clarke, the VP and\n>GM.\n>\n\nThe San Jose Sharks and Ottawa Senators ar\n\n###\n\n"

In [27]:
test['completion'][0]

' hockey'

We need to use the same separator following the prompt which we used during fine-tuning. In this case it is \n\n###\n\n. Since we're concerned with classification, we want the temperature to be as low as possible, and we only require one token completion to determine the prediction of the model.



In [28]:
fine_tune_job = client.fine_tuning.jobs.retrieve(fine_tuning_job.id)


In [29]:
ft_model = fine_tune_job.fine_tuned_model
# note that this calls the legacy completions api - https://platform.openai.com/docs/api-reference/completions

res=client.completions.create(
    model=ft_model,
    prompt=test['prompt'][0]+"\n\n###\n\n",
    max_tokens=1,
    temperature=0,


)
res.choices[0].text

' hockey'

#Generalization
Interestingly, our fine-tuned classifier is quite versatile. Despite being trained on emails to different mailing lists, it also successfully predicts tweets.

In [31]:
sample_hockey_tweet = """Thank you to the
@Canes
 and all you amazing Caniacs that have been so supportive! You guys are some of the best fans in the NHL without a doubt! Really excited to start this new chapter in my career with the
@DetroitRedWings
 !!"""
res = client.completions.create(model=ft_model, prompt=sample_hockey_tweet + '\n\n###\n\n', max_tokens=1, temperature=0, logprobs=2)
res.choices[0].text

' hockey'

In [32]:
sample_hockey_tweet = """With Tyler Bertuzzi and Andreas Athanasiou set to be reunited as teammates with the
@NHLBlackhawks
, will they bring back their pregame juggling routine?"""
res = client.completions.create(model=ft_model, prompt=sample_hockey_tweet + '\n\n###\n\n', max_tokens=1, temperature=0, logprobs=2)
res.choices[0].text

' hockey'